In [1]:
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.io
import plotly.express as px
from scipy import ndimage as ndi
import imageio
from scipy import signal
from pathlib import Path
import tensorflow_probability as tfp
import tensorflow as tf
from tqdm import tqdm  
from os import path
from myfunctions import event_separation , image_crop_save_gauss , image_crop_save , image_crop_negative, poi
import tifffile


## Input data from a folder ##

In [2]:
files_dir = r'C:\Users\roumba\Documents\Software\deep-events\original_data'
images_dir = '220915_mtstaygold_cos7_ZEISS_bf'
joined_path = os.path.join(files_dir, images_dir)
size=(2048,2048)
img,input_name,output_name,datacsv={},{},{},{}

for input_file in os.listdir(joined_path):
        joined_file_path = os.path.join(files_dir, images_dir, input_file)
        if '.tif' in input_file:
            date, cell_type, dye, bf_fl, index  = input_file.split('_')
            number, ome, tiff= index.split('.')
            img[int(number)-1] = Image.open(joined_file_path)
            input_name[int(number)-1]= f'points_{date}_{cell_type}_{bf_fl}_{number}'
            output_name[int(number)-1]= f'image_{date}_{cell_type}_{bf_fl}_{number}'
            print('Loaded image:'+input_file)
        else:
            labels, index, bf_fl  = input_file.split('_')
            datacsv[int(index)-1] = pd.read_csv(joined_file_path)
            print('Loaded csv:'+ input_file)
max_number=int(number)        


Loaded image:220915_cos7_mitostaygold_Brightfield_1.ome.tif
Loaded image:220915_cos7_mitostaygold_Brightfield_2.ome.tif
Loaded image:220915_cos7_mitostaygold_Brightfield_3.ome.tif
Loaded csv:labels_1_bf.csv
Loaded csv:labels_2_bf.csv
Loaded csv:labels_3_bf.csv


## Get all Gaussian Points of Interest ##

In [3]:
for x in range(max_number):
    csv=datacsv[x]
    im=img[x]
    for i in range(1,10,1):
        sigma=(i,i)
        s=sigma[0]
        in_name=f'{input_name[x]}_sigma{s}.tiff'
        framenum=im.n_frames
        poi(csv,in_name,sigma,size,framenum)

100%|██████████| 139/139 [00:36<00:00,  3.82it/s]


## Cut Images and Gaussian POIs ##

In [5]:
base_dir= r'C:\Users\roumba\Documents\Software\deep-events'
training_dir= r'C:\Users\roumba\Documents\Software\deep-events\training_data'
folder_name=images_dir+'_pos'
filepath= os.path.join(training_dir , folder_name)

for x in range(max_number):
    csv=datacsv[x]
    im=img[x]
    for i in range(1,10,1):
        sigma=(i,i)
        s=sigma[0]
        in_name=f'{input_name[x]}_sigma{s}.tiff'
        out_name=f'{output_name[x]}'
        out_name_g=f'{output_name[x]}_sigma{s}'
    
        list1=event_separation(csv)
        l=len(list1)
        image_crop_save(l,list1, csv, im, out_name,filepath)

        gauss=base_dir+f'\{in_name}'
        gauss_image=Image.open(gauss)
        image_crop_save_gauss(l,list1, csv, gauss_image, out_name_g,filepath)

ValueError: cannot convert float NaN to integer

## Cut Negative Frames ##

In [8]:
images_dir = '220915_mtstaygold_cos7_ZEISS_bf'
folder_name=images_dir+'_neg'
filepath= os.path.join(training_dir , folder_name)

for x in range(1,max_number):
    csv=datacsv[x]
    im=img[x]
    out_name=f'{output_name[x]}'
    in_name=f'{input_name[x]}_sigma{s}.tiff'
    image_crop_negative(l,list1, csv, im, out_name,filepath)

ValueError: cannot convert float NaN to integer